In [3]:
FEATURES = ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]  # (x,y,z,t,q)

In [2]:
from typing import Dict, Callable
import torch

In [4]:
import sys, os

# go two levels up:  .../Graphnet-Applications
repo_root = os.path.abspath("../..")
sys.path.append(repo_root)
from MyClasses.detector_pone import PONE

graphnet [MainProcess] WARNING  2026-01-11 06:17:14 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.


In [5]:
det = PONE(
    replace_with_identity=["charge", "dom_time", "dom_x", "dom_y", "dom_z"]
)

graphnet [MainProcess] INFO     2026-01-11 06:17:14 - PONE.__init__ - Writing log to logs/graphnet_20260111-061714.log


In [6]:
from graphnet.data.dataset.parquet.parquet_dataset import ParquetDataset
from graphnet.models.detector.prometheus  import  Prometheus
from graphnet.models.graphs  import  KNNGraph
from graphnet.models.graphs.nodes  import  NodesAsPulses

FEATURES = ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]  # (x,y,z,t,q)

data_representation = KNNGraph(
    detector=PONE(),
    node_definition=NodesAsPulses(),
    nb_nearest_neighbours=8,
    distance_as_edge_feature = True
)

dataset = ParquetDataset(
    path="/project/def-nahee/kbas/POM_Response_Parquet/merged" ,
    pulsemaps="features",
    truth_table="truth",
    features=FEATURES,
    truth=["energy"],
    data_representation = data_representation,
)



graphnet [MainProcess] WARNING  2026-01-11 06:17:14 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [MainProcess] WARNING  2026-01-11 06:17:14 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [MainProcess] WARNING  2026-01-11 06:17:14 - <module> - `graphnet.models.graphs` will be depricated soon. All functionality has been moved to `graphnet.models.data_representation`.


In [ ]:
dir(dataset)

In [ ]:
len(dataset)

In [ ]:
graph = dataset[696] 
graph

In [ ]:
dir(graph)

In [ ]:
graph.keys()          

In [ ]:
graph.energy          # truth burada

In [ ]:
graph.dom_x

In [ ]:
graph.event_no

In [ ]:
graph.track

In [ ]:
graph.num_nodes

In [ ]:
graph.graph_definition

In [ ]:
graph.x

In [ ]:
graph.x.shape

In [ ]:
graph.v_t

In [ ]:
graph.v_u

In [ ]:
graph.data_representation

In [ ]:
graph.charge

In [ ]:
graph.dataset_path

In [ ]:
graph.muon

In [ ]:
graph.features

In [ ]:
graph.corsika

In [ ]:
graph.neutrino

In [ ]:
graph.n_pulses

In [ ]:
graph.noise

In [ ]:
graph.dom_z

In [ ]:
graph.dom_y

In [ ]:
graph.dbang

In [ ]:
graph.muon_stopped

In [ ]:
graph.v_e

In [ ]:
graph.dom_time

In [ ]:
graph.edge_index

In [ ]:
graph.edge_index.shape

In [ ]:
graph.edge_attr

In [ ]:
graph.edge_attr.shape

In [ ]:
graph.node_attrs

In [ ]:
graph.num_edge_features

In [ ]:
graph.num_edges

In [ ]:
g = dataset[0]


print(torch.allclose(g.x[:,0], g.dom_x))
print(torch.allclose(g.x[:,1], g.dom_y))
print(torch.allclose(g.x[:,2], g.dom_z))
print(torch.allclose(g.x[:,3], g.dom_time))
print(torch.allclose(g.x[:,4], g.charge))

# from here we can see that the columns in `g.x` follow exactly the same order as your `FEATURES` list.

In [7]:
from graphnet.data.dataloader import DataLoader

In [ ]:
help(DataLoader)

In [8]:
loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    multiprocessing_context="spawn",  
    persistent_workers=True,         
)


# datasetleri onceden ayirip ayri ayri koysan iyi olcak gibi. merged icine train, test, val koy

In [9]:
it = iter(loader)


graphnet [SpawnProcess-1] WARNING  2026-01-11 06:17:25 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [SpawnProcess-1] WARNING  2026-01-11 06:17:25 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [SpawnProcess-1] WARNING  2026-01-11 06:17:28 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:17:30 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:17:31 - has_km3net_package - `km3net` not available. Some functionality may be missing.
graphnet [SpawnProcess-2] WARNING  2026-01-11 06:17:34 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.
graphnet [SpawnProcess-3] WARNING  2026-01-11 06:17:36 - has_icecube_package - `icecube` not available. Some functionality may be missing.

In [10]:
batch = next(it)


In [ ]:
batch

In [ ]:
dir(batch)

In [ ]:
batch.batch

In [ ]:
batch.edge_index

In [ ]:
batch.edge_index.shape

In [ ]:
batch.x

In [ ]:
# spawn ney ogren. burda spawn process

In [ ]:
from graphnet.models.components.layers import DynEdgeConv

In [ ]:
import torch.nn as nn
mlp = nn.Sequential(
    nn.Linear(10, 32),    ### 10 is 2*5
    nn.ReLU(),
    nn.Linear(32, 256)   
)

In [ ]:
layer = DynEdgeConv(
    nn=mlp,
    nb_neighbors=8,  
    features_subset=None,
    aggr = "add"
)

In [ ]:
x1, edge_index1 = layer(batch.x, batch.edge_index, batch=batch.batch)

In [ ]:
batch.x.shape

In [ ]:
x1.shape

In [ ]:
batch.edge_index.shape
# bu ok degil

In [ ]:
edge_index1.shape
# bu ok 

## neden :(

Koduna baktığımda her şey genel olarak mantıklı görünüyor. Yani KNNGraph ile baştan itibaren nb_nearest_neighbours=8 olarak belirlemişsin ve bu ayarlar da DynEdgeConv layer’ına düzgünce yansıtılmış. Buna rağmen edge sayısında fazlalık devam ediyorsa, bu durumda veri setinin ilk oluşturulduğu aşamada eklenmiş ekstra kenarlar olabileceğini düşünüyorum.

Yani aslında kodun mantığı doğru ama KNNGraph oluşturulurken veri setinin bir şekilde zaten fazladan edge ile gelmiş olması mümkün. Belki de distance_as_edge_feature=True parametresi ya da NodesAsPulses tanımı sırasında beklenmedik ekstra komşuluklar oluşmuş olabilir.

Bu durumda yapılabilecek şeylerden biri, batch.edge_index’i layer’dan önce bir kez kontrol edip, fazladan kenarları elle temizlemeyi denemek olabilir. Ya da KNNGraph tanımında veya PONE içindeki ayarlarda fazladan bir şeyin eklenmediğinden emin olmak için ayarları gözden geçirebilirsin.

Kısacası, kodun doğru yolda, sadece veri setinin baştan fazla edge ile gelmesi olasılığını bir kontrol etmek faydalı olur.

In [11]:
from graphnet.models.gnn  import  DynEdge
backbone = DynEdge(nb_inputs=5, nb_neighbours=8, 
                      # features_subset 
                      # dynedge_layer_sizes 
                      # post_processing_layer_sizes 
                      # readout_layer_sizes
                      global_pooling_schemes = ["min","max","mean","sum"],
                      add_global_variables_after_pooling = True, 
                      add_norm_layer = False, 
                      skip_readout = False                      
                     )

In [ ]:
backbone.nb_outputs

In [ ]:
backbone.nb_inputs

In [12]:
model = backbone(batch)

In [ ]:
model

In [ ]:
model.shape

In [ ]:
# think about initialization, relu-gelu, homophily, pmt/dom location, normalization layer, learning rate, loss metrics, regularization
# also check this folder: src/graphnet/training
# adam optimizer'dan nerede bahsediliyor?

In [ ]:
# in the article, target is the energy itself. then, in the loss function, logarithm of both truth and pred is used.

# burdan sonrasinin source code oku ve anlamaya calis cnm

In [13]:
from graphnet.models.task import StandardLearnedTask
from graphnet.utilities.maths import eps_like
from torch import Tensor

In [14]:
def logarithm(E: torch.Tensor) -> torch.Tensor:
    """
    Forward transform: E -> log10(E)
    Safety: clamps E to >= eps to avoid log10(0) / log10(negative).
    """
    E_safe = torch.clamp(E, min=eps_like(E))
    return torch.log10(E_safe)

def exponential(t: torch.Tensor) -> torch.Tensor:
    """
    Inverse transform: log10(E) -> E
    """
    return torch.pow(10.0, t)

interval = (1e1, 1e8) 


In [15]:
len(interval)

2

In [16]:
class DepositedEnergyLog10Task(StandardLearnedTask):
    default_target_labels = ["energy"]
    default_prediction_labels = ["log10_energy_pred"]
    nb_inputs = 1

    def _forward(self, x: Tensor) -> Tensor:
        return x[:, :1]

   

In [17]:
from graphnet.training.loss_functions import LogCoshLoss

In [18]:
task = DepositedEnergyLog10Task(
    hidden_size = backbone.nb_outputs,
    loss_function = LogCoshLoss(),
    target_labels =["energy"],
    prediction_labels = ["log_energy"],
    transform_prediction_and_target = None,
    transform_target = logarithm,
    transform_inference = exponential,
    transform_support = interval,
    loss_weight = None)

# _validate_and_set_transforms calistirmana gerek yok. init esnasinda calistitiliyor.

In [19]:
task._transform_target

<function __main__.logarithm(E: torch.Tensor) -> torch.Tensor>

In [20]:
task._transform_prediction_inference

<function __main__.exponential(t: torch.Tensor) -> torch.Tensor>

In [21]:
# if i call this class, these will run in the given order:
# 1- _regularisation_loss = 0

In [22]:
task._regularisation_loss = 0

In [23]:
# 2- _affine(x)

In [24]:
model = task._affine(model)

In [25]:
model

tensor([[202963.2812],
        [206201.3438],
        [207985.1875],
        [204048.9688]], grad_fn=<AddmmBackward0>)

In [26]:
# 3- _forward(x): no change 

In [27]:
model = task._forward(model)

In [28]:
model

tensor([[202963.2812],
        [206201.3438],
        [207985.1875],
        [204048.9688]], grad_fn=<SliceBackward0>)

In [29]:
# 4- _transform_prediction(x): no change 

In [30]:
model = task._transform_prediction(model)

In [31]:
model

tensor([[202963.2812],
        [206201.3438],
        [207985.1875],
        [204048.9688]], grad_fn=<SliceBackward0>)

In [32]:
# now, it is time to calculate the loss. if i call task.compute_loss(pred, data):

In [33]:
# pred is the model itself

In [34]:
model

tensor([[202963.2812],
        [206201.3438],
        [207985.1875],
        [204048.9688]], grad_fn=<SliceBackward0>)

In [35]:
# data is the batch
# pred is the model

In [36]:
# 1- it checks self._target_labels, and takes it

In [37]:
task._target_labels

['energy']

In [38]:
target = torch.stack(
            [batch[label] for label in task._target_labels], dim=1
        )

In [39]:
target

tensor([[ 116.7452],
        [ 184.3812],
        [1267.7957],
        [ 104.7204]], dtype=torch.float64)

In [40]:
# 2- it does this: self._transform_target(target) 

In [41]:
target = task._transform_target(target)

In [42]:
target

tensor([[2.0672],
        [2.2657],
        [3.1030],
        [2.0200]], dtype=torch.float64)

In [43]:
weights = None

In [44]:
# 3- loss is calculated:

# loss = (
#             self._loss_function(pred, target, weights=weights)
#             + self._regularisation_loss
#         )

In [45]:
task._loss_function

LogCoshLoss(
  LogCoshLoss(
  {
      'arguments': {
      },
  })
)

In [46]:
task._loss_function(prediction=model, target=target)

tensor(205296.6382, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [47]:
task._regularisation_loss

0

In [50]:
loss = (
            task._loss_function(prediction=model, target=target)
            + task._regularisation_loss
        )

In [51]:
loss

tensor(205296.6382, dtype=torch.float64, grad_fn=<AddBackward0>)